In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
house = pd.read_csv("../input/train.csv")
house_test = pd.read_csv("../input/test.csv")
print(house.shape)
print(house_test.shape)

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',82)
house

In [ ]:
house.corr().SalePrice

In [ ]:
#From this it is clear that OverallQual,TotalBsmtSF,1stFlrSF,GRLivArea,GarageCars,GarageArea affects the saleprice of the house
house.mean().SalePrice

In [ ]:
#Outlier treatment in most needed column

    quad_1 = house['GarageArea'].describe()['25%']
    quad_3 = house['GarageArea'].describe()['75%']
    median = df_in[col_name].describe()['median']
    iq_range = quad3-quad1     
    rest_low  = quad_1-(1.5*iq_range)
    rest_high = quad_3+(1.5*iq_range)
    house.loc[(house['GarageArea'] <= rest_low) | (house['GarageArea'] >= rest_high),'GarageArea'] = median
    

In [ ]:
#Replacing target variable in test dataset with mean of train
house_test['SalePrice']  = 180921.00#giving the mean of test dataset
house_test.head()

In [ ]:
from sklearn import linear_model
from sklearn.cross_validation import train_test_split

In [ ]:
reg=linear_model.LinearRegression()

In [ ]:
x_train = house[['OverallQual','TotalBsmtSF','1stFlrSF','GrLivArea','GarageCars','GarageArea']]
y_train = house['SalePrice']

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
x_test = house_test[['OverallQual','TotalBsmtSF','1stFlrSF','GrLivArea','GarageCars','GarageArea']]
y_test = house_test['SalePrice']

In [ ]:
x_test.count()

In [ ]:
#As we can see there are some missing values here so the next step is to replace these missing values
x_test.describe()

In [ ]:
x_test['TotalBsmtSF'].fillna(1046,inplace=True)
x_test['GarageCars'].fillna(2,inplace=True)
x_test['GarageArea'].fillna(472.76,inplace=True)

In [ ]:
y_test=reg.predict(x_test)

In [ ]:
house_test['SalePrice'] = y_test
house_test.head()
#Last column of Sale Price is replaced by linear regression predicted values 

In [ ]:
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
from scipy import stats
from scipy.stats import norm, skew

In [ ]:
sns.distplot(house['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(house['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(house['SalePrice'], plot=plt)
plt.show()

In [ ]:
#Data is Right skewed log transformation would be a better approach
house["SalePrice"] = np.log1p(house["SalePrice"])

#Check the new distribution 
sns.distplot(house['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(house['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(house['SalePrice'], plot=plt)
plt.show()

In [ ]:
sns.regplot(x="OverallQual", y="SalePrice", data=house)

In [ ]:
sns.regplot(x="TotalBsmtSF",y="SalePrice",data=house)

In [ ]:
sns.regplot(x="1stFlrSF",y="SalePrice",data=house)

In [ ]:
sns.regplot(x="GrLivArea",y="SalePrice",data=house)

In [ ]:
sns.regplot(x="GarageCars",y="SalePrice",data=house)

In [ ]:
sns.regplot(x="GarageArea",y="SalePrice",data=house)

In [ ]:
#Drawing Linear Regression Residual vs fitted plot for training and test dataset 
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = sns.residplot(reg.predict(x_train), y_train, lowess=True,scatter_kws={'alpha': 0.5}, line_kws={'color': 'black', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted - Train_ Sample ')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')
plt.show()


plot_lm_2 = plt.figure(1)
plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0] = sns.residplot(reg.predict(x_test), y_test,lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'green', 'lw': 1, 'alpha': 0.8})

plot_lm_2.axes[0].set_title('Residuals vs Fitted - Test_Sample ')
plot_lm_2.axes[0].set_xlabel('Fitted values')
plot_lm_2.axes[0].set_ylabel('Residuals')
plt.show()

In [ ]:
#checking R**2 value

print("Mean squared error in house_train_dataset:",np.mean((reg.predict(x_train) - y_train) ** 2))
print('R² of train:',reg.score(x_train, y_train))

In [ ]:
#Now trying polynomial regression as it might be a better fit
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=2)
x_poly_train = poly_reg.fit_transform(x_train)
x_poly_test  = poly_reg.fit_transform(x_test)

In [ ]:
reg_2 = linear_model.LinearRegression()

In [ ]:
reg_2.fit(x_poly_train,y_train)

In [ ]:
plot_pm_1 = plt.figure(1)
plot_pm_1.set_figheight(8)
plot_pm_1.set_figwidth(12)

plot_pm_1.axes[0] = sns.residplot(reg_2.predict(x_poly_train), y_train, lowess=True,scatter_kws={'alpha': 0.5}, line_kws={'color': 'black', 'lw': 1, 'alpha': 0.8})

plot_pm_1.axes[0].set_title('Residuals vs Fitted poly- Train_Sample ')
plot_pm_1.axes[0].set_xlabel('Fitted values')
plot_pm_1.axes[0].set_ylabel('Residuals')
plt.show()


plot_pm_2 = plt.figure(1)
plot_pm_2.set_figheight(8)
plot_pm_2.set_figwidth(12)

plot_pm_2.axes[0] = sns.residplot(reg_2.predict(x_poly_test), y_test,lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'green', 'lw': 1, 'alpha': 0.8})

plot_pm_2.axes[0].set_title('Residuals vs Fitted poly - Test_Sample ')
plot_pm_2.axes[0].set_xlabel('Fitted values')
plot_pm_2.axes[0].set_ylabel('Residuals')
plt.show()

In [ ]:
reg_2.score(x_poly_train,y_train)

In [ ]:
#Polynomial regression provides better fitting than linear regression
#One more model could be a polynomial regression of degree 3
poly_reg2 = PolynomialFeatures(degree=3)
x_poly_train2 = poly_reg2.fit_transform(x_train)
x_poly_test2  = poly_reg2.fit_transform(x_test)


In [ ]:
reg_2.fit(x_poly_train2,y_train)

In [ ]:
plot_pm_1 = plt.figure(1)
plot_pm_1.set_figheight(8)
plot_pm_1.set_figwidth(12)

plot_pm_1.axes[0] = sns.residplot(reg_2.predict(x_poly_train2), y_train, lowess=True,scatter_kws={'alpha': 0.5}, line_kws={'color': 'black', 'lw': 1, 'alpha': 0.8})

plot_pm_1.axes[0].set_title('Residuals vs Fitted poly2- Train_Sample ')
plot_pm_1.axes[0].set_xlabel('Fitted values')
plot_pm_1.axes[0].set_ylabel('Residuals')
plt.show()


plot_pm_2 = plt.figure(1)
plot_pm_2.set_figheight(8)
plot_pm_2.set_figwidth(12)

plot_pm_2.axes[0] = sns.residplot(reg_2.predict(x_poly_test2), y_test,lowess=True, scatter_kws={'alpha': 0.5}, line_kws={'color': 'green', 'lw': 1, 'alpha': 0.8})

plot_pm_2.axes[0].set_title('Residuals vs Fitted poly2 - Test_Sample ')
plot_pm_2.axes[0].set_xlabel('Fitted values')
plot_pm_2.axes[0].set_ylabel('Residuals')
plt.show()

In [ ]:
reg_2.score(x_poly_train2,y_train)